In [1]:
import numpy as np 
import pandas as pd
import openpyxl
#import os
from ortools.linear_solver import pywraplp
print("Bibliotecas carregadas")


Bibliotecas carregadas


In [11]:
def read_data(DataFile):
    xlsLoad = pd.ExcelFile(DataFile)
    Bus = pd.read_excel(xlsLoad, 'bus').set_index('id')
    Gen = pd.read_excel(xlsLoad, 'gen').set_index('id')
    Line = pd.read_excel(xlsLoad, 'line').set_index('id')
    #id should be a column with unique information. 
    
    return Bus, Gen, Line

print(read_data('RTS_Data.xlsx')[0])

    Original Bus #  Area  Rated kV  P-load  Q-load
id                                                
1              101     1       138     108      22
2              102     1       138      97      20
3              103     1       138     180      37
4              104     1       138      74      15
5              105     1       138      71      14
6              106     1       138     136      28
7              107     1       138     125      25
8              108     1       138     171      35
9              109     1       138     175      36
10             110     1       138     195      40
11             111     1       230       0       0
12             112     1       230       0       0
13             113     1       230     265      54
14             114     1       230     194      39
15             115     1       230     317      64
16             116     1       230     100      20
17             117     1       230       0       0
18             118     1       

In [ ]:
def DCOPF(Bus, Gen, Line):
    # define a pyomo model
    solver = pywraplp.Solver('simple_lp_program', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    N=Bus.index
    G=Gen.index
    K=Line.index
    
    # 
    p = [solver.NumVar(lb=0, ub=solver.infinity) for _ in G]
    f = [solver.NumVar(lb=0, ub=solver.infinity) for _ in K]
    theta = [solver.NumVar(lb=0, ub=solver.infinity) for _ in N]
    
    # Define the objective function
    objective = solver.objective()
    for g in G:
        objective.SetCoefficient(p[g], Gen.loc[g, 'vCost1'])
    
    # define model constraints
    model.gen_max = ConstraintList()
    model.flow = ConstraintList()
    model.flow_max = ConstraintList()
    model.node_balance = ConstraintList()
    model.slack=ConstraintList()
    
    # Add the constraints to the model
      
    # Generation maximum Limits:
    for g in G:
        model.gen_max.add(model.p[g]<=Gen.loc[g,'CapHigh'])
    
    # Power flow equations and limits
    for k in K:
        model.flow.add(model.f[k]==Line.loc[k,'B_dc (MW)']*
                       (model.theta[Line.loc[k,'from']]-model.theta[Line.loc[k,'to']]))
        model.flow_max.add(model.f[k]<=Line.loc[k,'CAPA'])
        model.flow_max.add(model.f[k]>=-Line.loc[k,'CAPA'])
    
    # Slack bus constraint
    model.slack.add(model.theta[1]==0)
    
    # Node balance constraints
    
    for n in N:
        demand = Bus.loc[n,'P-load']
        inflow = 0
        outflow = 0
        localgen = 0
        for k in K:
            if Line.loc[k,'to']==n:
                inflow += model.f[k]
            if Line.loc[k,'from']==n:
                outflow += model.f[k]
        for g in G:
            if Gen.loc[g,'node']==n:
                localgen += model.p[g]
        model.node_balance.add(localgen+inflow-outflow==demand)
    return model
    
                




In [6]:
# Read Data
#print(os.getcwd())
Bus, Gen, Line = read_data('RTS_Data.xlsx')
print("Data was read successfully.")
N=Bus.index
G=Gen.index
K=Line.index

# Construct Model
model = DCOPF(Bus, Gen, Line)
print("Construction of model complete.")

# Solving steps

# Identify the solver
opt = SolverFactory('glpk')

# Tell the model that you need the dual values too!
model.dual = Suffix(direction=Suffix.IMPORT)

# Solve the problem
Solution=opt.solve(model)

# Report the summary of solve
Solution.write(num=1)

# Report the output
for g in G:
    print('Generator ',g,'Output (MW):', model.p[g].value)
    
for k in K:
    print('Line ',k,'Flow (MW):', model.f[k].value)

for n in N:
    print('LMP for node ', n, 'in ($/MWh) is: ', model.dual[model.node_balance[n]])
    
for k in K:
    print('Flowgate price for line', k, 'in ($/MWh) in both directions are: '
          , model.dual[model.flow_max[2*k-1]],' and ', model.dual[model.flow_max[2*k]])


Data was read successfully.
Construction of model complete.
    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'